# 1. Establish communication with the card

## 1.1. Find the smartcard reader

In [50]:
from applets.helpers import get_connection, AppletBase

In [51]:
connection = get_connection()
if connection is None:
    print("Failed to open a connection. No cardreaders?")

## 1.2. Power on the card and get `ATR`
`ATR` - Answer To Reset, bytes retured by the card when it is powered on.<br>
It helps to determine communication protocol and frequency we should choose.

In [52]:
atr=bytes(connection.getATR())
print(atr.hex())

3bdc18ff8191fe1fc38073c821136605036351000250


## 1.3. Select the applet by AID
JavaCard can have multiple applets installed. They are like very isolated "apps".<br>
They can't talk to each other and only one can be active at any time.

In [53]:
from applets.teapot import Teapot

class MemoryCard(Teapot):
    def __init__(self, connection=None):
        super().__init__(connection)
        self.AID = "B00B5111CB01"

    def get_random(self):
        return self.request("B0B10000")

In [54]:
app = MemoryCard(connection)
app.select()

b''

In [55]:
app.get_random().hex()

'054c29537358ed7c42c8277749381c4a6220460ee93634523a0a6e43fbef9b76'

# 2. Establish secure communication

At the moment we use a homebrew version, I need to understand and implement Noise in the near future.

[Noise protocol](https://noiseprotocol.org/noise.html) should be with the following primitives:
- secp256k1 as signature scheme
- AES CBC with random IV as a symmetric cipher
- SHA256 as a hash function, HMAC_SHA256 as authentication code

AES CBC because we have hardware implementation on the card, so we can use the Key class that is more secure than byte array. HMAC sucks here though...

Generally a shared secret is split to 2 different secrets for encryption and decryption. This allows us to use AES in counter mode where the counter is the same for both host and the card - they use different keys so it's ok.

Description of the current protocol:
1. Get the static public key from the card. If it is already known - proceed with step 2
2. Generate a random key pair and send corresponding pubkey in uncompressed form to the card: `<data:pub_uncompressed>`
3. Card will return its fresh session public key, hmac of this public key using shared secret and signature: `<data:session_pubkey_card><hmac(dec,data)><sig(static_pubkey_card,data|hmac)>`
4. Verify the signature against card's pubkey
5. Calculate a pair of shared secrets from `enc=SHA256('host'|ECDH(d,session_pubkey_card))`, `dec=SHA256('card'|ECDH(d,session_pubkey_card)`
6. Verify that `hmac==HMAC_SHA256(dec,session_pubkey_card)`
7. Set counter to zero `iv=0`, use `dec` for decryption, `enc` for enctyption. Increase the counter on every request, reestablish communication if the counter reaches max value (16 bytes, shouldn't happen in most cases)
8. All secure messages now should be in the form `<ct:aes(enc,iv,command|data)><hmac_sha256(enc,ct)>`, `iv` is not transmitted but increased for every new command. Returned data will be of the same form but using the `dec` key. `iv` will be the same.

Maximum size of the APDU is 256 bytes. Maximum size of the mnemonic is `9*24-1=215` bytes plus length and command and rounding to `16` bytes, plus `32`-byte hmac - we are at `256` bytes... We can send at max `255` bytes.

So to store and retrieve large mnemonic we need to split it to two pieces, and provide a function to set part of the data array. Therefore for secret storage we have to make an API: 
- `set_data<data>` / `get_data` - for short mnemonics
- `set_data_length<len>` - should be called first, erases existing data, 
- `set_data_part <start><data>` - stores part of the data with an offset
- `get_data_length`
- `get_data_part <start><len>` - gets part of the data from `start` with `len`


## 2.1. Get public key from the card

In [56]:
# embit library: https://github.com/diybitcoinhardware/embit
# works with python3 and micropython
from embit.ec import PrivateKey, PublicKey, secp256k1, Signature

In [57]:
# get pubkey - private key is generated when applet is installed
sec = app.request("B0B2000000")
card_pubkey = PublicKey.parse(sec)
card_pubkey

PublicKey(04456bfaaa634704ed87223b6d6d0476e2ffb3112a005d2fd742782f8e535a392957466d7a7bed35a7f377be5a1417a1c9a6e7c1eb9285e5551bf0ff2473822c42)

## 2.2. Select a random secret on the host

In [58]:
import os
import hashlib
secret = os.urandom(32)

In [59]:
host_prv = PrivateKey(secret)
host_pub = host_prv.get_public_key()
host_pub.compressed = False
data = bytes([65])+host_pub.sec()
data.hex()

'4104a90903701a56a18cc6a029dfee1c77b28c2fff21ff08fba6e55ac0dfb790d06dffc5878c2bca05637bc5ed84a2b9a07369be01e34702c1245391925a86aa2b17'

In [65]:
# ugly copy...
pub = PublicKey.parse(card_pubkey.sec())
secp256k1.ec_pubkey_tweak_mul(pub._point, secret)
shared_secret = pub.sec()[1:33]
host_key = hashlib.sha256(b'host'+shared_secret).digest()
card_key = hashlib.sha256(b'card'+shared_secret).digest()
shared_hash = hashlib.sha256(host_key+card_key).digest()
print("Shared secret:", shared_secret.hex())
print("Sha256 of the secrets:", shared_hash.hex())

Shared secret: b74943449e788c519fcb494844fcbb0359a94fc36d34d02d77829740d7a0b072
Sha256 of the secrets: e23b2486a4dced89ac09a46bec75a1a1d2b0ff3ca8c90c138535b095905c5404


In [61]:
res = app.request("B0B30000"+data.hex())
print(res.hex())

e23b2486a4dced89ac09a46bec75a1a1d2b0ff3ca8c90c138535b095905c5404fae9360257f43bbaf29eb20c9362e6d147bc9c640a700e1741f714ed81ff14883046022100f878b0ec3eaf7ded141e6d695cac572ba00172462a5fc54de68f2e7700d52296022100b7714b28d8fefad5d6ee1ce425ba3eacf4ec186d274c1f6869374c5b93face3b


In [66]:
import hmac
recv_hmac = res[32:64]
h = hmac.new(card_key, digestmod='sha256')
h.update(res[:32])
expected_hmac = h.digest()
if expected_hmac != recv_hmac:
    raise RuntimeError("Wrong HMAC. Got %s, expected %s"%(recv_hmac.hex(),expected_hmac.hex()))
sig = Signature.parse(res[64:])
# card doesn't follow low s rule, so we need to normalize
sig._sig = secp256k1.ecdsa_signature_normalize(sig._sig)
if not card_pubkey.verify(sig, hashlib.sha256(res[:64]).digest()):
    raise RuntimeError("Signature is invalid")
if res[:32] == shared_hash:
    print("Yey! We have shared secrets now!")
else:
    print("Meh... Something didn't work? Card returned %s, we have %s" % (res[:32].hex(), shared_hash.hex()))

Yey! We have a shared secret now!


In [67]:
# alternative ecdh:
res = app.request("B0B40000"+data.hex())
print(res.hex())

043495f0087eaf759639948dd17db552dbd55c2163dcc62008f0e87d176ba84cb3151fabc183f6b37b4e6d4abf5f73e36d6ff127edde2e5a190cec7bbec030a8e5023944648c67e21729155691e4dac7bc078e789a84069e276c37ff1cf465a4dd30440220433cc3bb1cc3a7e428eb0046e61ff15ff3c2ab838104adb71b996495a65ac63d022078e5914fbd1839d4aa4bd3fe7e69604775fa7bb2208b92584e5524b93374cb22


In [68]:
pub = PublicKey.parse(res[:65])
secp256k1.ec_pubkey_tweak_mul(pub._point, secret)
shared_secret = pub.sec()[1:33]
host_key = hashlib.sha256(b'host'+shared_secret).digest()
card_key = hashlib.sha256(b'card'+shared_secret).digest()
shared_hash = hashlib.sha256(host_key+card_key).digest()
print("Shared secret:", shared_secret.hex())
print("Sha256 of the secret:", shared_hash.hex())

Shared secret: e0026ed304a0f282786e977fc446c906c570b6b1dd12d6ac443ccc8bb37f152a
Sha256 of the secret: 8e1ff73c49ed9dd184f37aff88e35cc71cbe663344998d00b37515ddd53132ff


In [69]:
recv_hmac = res[65:97]
h = hmac.new(card_key, digestmod='sha256')
h.update(res[:65])
expected_hmac = h.digest()
if expected_hmac != recv_hmac:
    raise RuntimeError("Wrong HMAC. Got %s, expected %s"%(recv_hmac.hex(),expected_hmac.hex()))
sig = Signature.parse(res[97:])
# card doesn't follow low s rule, so we need to normalize
sig._sig = secp256k1.ecdsa_signature_normalize(sig._sig)
if not card_pubkey.verify(sig, hashlib.sha256(res[:97]).digest()):
    raise RuntimeError("Signature is invalid")
print("Yey! We have a shared secret now!")

Yey! We have a shared secret now!


In [44]:
# get random data
rand = app.request("B0B2000000")
rand.hex()

'04d380946cfa081f4e3fc48f9adc540c9959c016a291145b537747cceb33a497e593803a1399bcd3a0ff62a78fbd378294d1c7c167a1573904bcd40c018dc1124a'

In [ ]:
# static data for any other APDU
data = app.request("B0A1000000")
data

In [ ]:
(bytes([len(data)])+data).hex()

In [ ]:
# static data for any other APDU
c = app.request("B0A40000"+(bytes([len(data)])+data).hex())
c

In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import hmac

def decrypt(c,secret):
    recv_hmac = c[-32:]
    iv = c[:16]
    ct = c[16:-32]
    h = hmac.new(shared_secret, digestmod='sha256')
    h.update(iv)
    h.update(ct)
    expected_hmac = h.digest()
    if expected_hmac != recv_hmac:
        raise RuntimeError("Wrong HMAC. Got %s, expected %s"%(recv_hmac.hex(),expected_hmac.hex()))
    backend = default_backend()
    cipher = Cipher(algorithms.AES(secret), modes.CBC(iv), backend=backend)
    decryptor = cipher.decryptor()
    # remove \x80... padding
    plain = decryptor.update(ct)+decryptor.finalize()
    return b"".join(plain.split(b"\x80")[:-1])

decrypt(c, shared_secret)

In [70]:
connection.disconnect()